In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkStreamTrigger")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/05/24 10:57:04 WARN Utils: Your hostname, Hocines-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.164 instead (on interface en0)
22/05/24 10:57:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/24 10:57:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

22/05/24 10:57:06 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
lineStreamDf.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
echoOnconsole = lineStreamDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .trigger(processingTime='10 seconds')\
                .start()


22/05/24 10:57:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/y4/h7s1_2k959z8nfw5xqv11nmr0000gn/T/temporary-076984d1-c50a-446c-82f6-7b8fcfc104a8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/24 10:57:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
|    8|
|    9|
|   10|
+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|value|
+-----+
|  ONE|
|  TWO|
+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+-----+
|value|
+-----+
|THREE|
| FOUR|
+-----+



In [6]:
# query id
echoOnconsole.id

'fe3bfd32-22d6-482c-8aa4-e2601268e33c'

In [7]:
# auto generated or user given
echoOnconsole.name 

In [8]:
# if any exception
echoOnconsole.exception

<bound method StreamingQuery.exception of <pyspark.sql.streaming.StreamingQuery object at 0x7fbe787093d0>>

In [9]:
echoOnconsole.recentProgress


[{'id': 'fe3bfd32-22d6-482c-8aa4-e2601268e33c',
  'runId': '8a716316-2031-4d06-8959-4ece111d0cf6',
  'name': None,
  'timestamp': '2022-05-24T15:57:11.049Z',
  'batchId': 0,
  'numInputRows': 0,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 0.0,
  'durationMs': {'addBatch': 1205,
   'getBatch': 2,
   'latestOffset': 0,
   'queryPlanning': 255,
   'triggerExecution': 1578,
   'walCommit': 56},
  'stateOperators': [],
  'sources': [{'description': 'TextSocketV2[host: localhost, port: 9999]',
    'startOffset': None,
    'endOffset': -1,
    'latestOffset': -1,
    'numInputRows': 0,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 0.0}],
  'sink': {'description': 'org.apache.spark.sql.execution.streaming.ConsoleTable$@450d75bf',
   'numOutputRows': 0}},
 {'id': 'fe3bfd32-22d6-482c-8aa4-e2601268e33c',
  'runId': '8a716316-2031-4d06-8959-4ece111d0cf6',
  'name': None,
  'timestamp': '2022-05-24T15:57:30.004Z',
  'batchId': 1,
  'numInputRows': 10,
  'inputRowsPerSeco

In [ ]:
# await for termination 
# block next operations until any termination
echoOnconsole.awaitTermination()
print("await after") # this statement won't be executed until stream close
# now you have to go to server, stop the server using Ctrl + C